In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import pandas as pd
from implementations import *

In [2]:
#TODO move this to a helpers folder and also why was a helper not provided anywhere?
def load_csv_data(data_path):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int64) #check if int 64 precision was actually needed
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1

    return yb, input_data, ids

In [3]:
df = pd.read_csv('train.csv')


In [4]:
print(df.columns[0])

Id


In [5]:
df.columns

Index(['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'],
      dtype='object')

In [6]:
df['PRI_jet_num']

0         2
1         1
2         1
3         0
4         0
         ..
249995    0
249996    0
249997    1
249998    0
249999    0
Name: PRI_jet_num, Length: 250000, dtype: int64

### Code start 

In [7]:
import datetime

DATA_TRAIN_PATH = 'train.csv'
DATA_TEST_PATH = 'test.csv'
y_train, tX_train, ids_train = load_csv_data(DATA_TRAIN_PATH)
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)


In [8]:
def clean_data(data):
    data_cleaned = data
    
    data_cleaned[data_cleaned == -999] = np.NaN
    
    #replace NaN's by mean of columns
    medians = np.nanmedian(data_cleaned, axis=0)
    print(medians.shape)
    sq_std = np.std(data_cleaned, axis=0) **2
    inds = np.where(np.isnan(data_cleaned))
    data_cleaned[inds] = np.take(medians, inds[1])
    
    #standardize the columns 
    data_cleaned = (data_cleaned - medians)  / sq_std 
    
    
    
    return data 

In [9]:
tX_train_clean = clean_data(tX_train)
tX_test_clean = clean_data(tX_test)

(30,)
(30,)


In [10]:
print(tX_train_clean[:, 22])

[2. 1. 1. ... 1. 0. 0.]


In [11]:
tX_test_clean.mean()

43.34846886732912

In [12]:
print(y_train)

[ 1. -1. -1. ...  1. -1. -1.]


In [14]:
w, loss = logistic_regression(y_train, tX_train_clean, initial_w = np.zeros((tX_train_clean.shape[1]+1, 1)), max_iters=400, gamma = 0.5)

MemoryError: Unable to allocate 466. GiB for an array with shape (250000, 250000) and data type float64

In [14]:
w.shape

(30,)

In [15]:
pred = tX_test_clean.dot(w)

#pred = (pred - pred.min()) / (pred.max() - pred.min())

In [16]:
pred.shape

(568238,)

In [17]:
pred[pred < 0] = -1
pred[pred > 0] = 1

In [18]:
pred.shape

(568238,)

In [19]:
pred = np.array(pred,  dtype=np.int64)

In [20]:
pred.shape

(568238,)

In [21]:
ids_test = ids_test[0]

In [22]:
prediction = np.c_[ids_test, pred]

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 568238

In [88]:
from numpy import savetxt
savetxt('prediction.csv', prediction, delimiter=',')

In [92]:
import csv 
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, "w") as csvfile:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({"Id": int(r1), "Prediction": int(r2)})


In [93]:
create_csv_submission(ids_test, pred, "pred1.csv")

dtype('int64')

In [78]:
ids_test.shape

(1, 568238)

In [69]:
predictions = np.concatenate((ids_test[0], pred), axis = 1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [68]:
predictions.shape

(1136476,)

In [28]:
y_test

array([1., 1., 1., ..., 1., 1., 1.])

In [16]:
errors = np.sum(np.abs((y_test - rounded_preds)))

In [17]:
tot = y_test.shape[0]

In [18]:
(tot - errors) / tot

0.9823190282944823

In [19]:
y_test.dtype

dtype('float64')

In [20]:
def real_positives(pred, act):
    tot = 0
    good = 0
    for p,a in zip(pred,act):
        if(a == 1.0):
            tot += 1
            if(p == 1.0):
                good += 1
    return good / tot
                
            
def real_negatives(pred,act):
    tot = 1
    good = 0
    for p,a in zip(pred,act):
        if(a == 0.0):
            tot += 1
            if(p == 0.0):
                good += 1
    return good, tot, good / tot
        
    

In [21]:
real_positives(rounded_preds, y_test)

0.9823190282944823

In [22]:
real_negatives(rounded_preds, y_test)

(0, 1, 0.0)

In [23]:
zero_y_test = y_test[y_test == 0.0]

In [24]:
print(zero_y_test)

[]


In [23]:
y_test.sum()

568238.0

In [26]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [28]:
xtr, xte, ytr, yte = split_data(tX_train, y_train, ratio=0.75, seed= 1)

In [29]:
print(xtr.shape)

(187500, 30)


In [30]:
print(tX_train.shape)

(250000, 30)


In [31]:
w, loss = log(ytr, xtr, 0)

In [37]:
pred = xte.dot(w)
pred = (pred - pred.min()) / (pred.max() - pred.min())

In [38]:
rounded_preds = np.round(pred, 0)

In [39]:
errors = np.sum(np.abs((yte - rounded_preds)))

In [40]:
tot = y_test.shape[0]

In [41]:
(tot - errors) / tot

0.8882915257339354